# 8장 컴퓨터 비전을 위한 딥러닝

## 8.1 합성곱 신경망 소개

In [ ]:
# 간단한 컨브넷 만들기

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28,28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

- Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) zㅡ기의 랭크-3 텐서
- 마지막 Conv2D 층의 출력 크기는 (3, 3, 128)

In [ ]:
# MNIST 이미지에서 컨브넷 훈련하기

from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"테스트 정확도: {test_acc: .3f}")

### 8.1.1 합성곱 연산

- Dense층은 전역 패턴을 학습하지만, 합성곱 층은 지역 패턴을 학습함

이 특징은 컨브넷에 두 가지 성질을 제공함
1. 학습된 패턴은 평행 이동 불변성(translation invariant)을 가짐.
    - 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면, 다른 곳에서도 이 패턴을 인식할 수 있음.
2. 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있음.
    - 첫 번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습하고, 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식이다.

- 합성곱 연산은 특성 맵(feature map)이라고 부르는 랭크-3 텐서에 적용된다. 텐서는 2개의 공간 축(높이와 너비)와 깊이 축(=채널 축)으로 구성 됨.